In [14]:
import pandas as pd
import json
from os import listdir
from os.path import isfile, join

In [15]:
englishMasculineWordsList = open('./data/gendered_words_lists/en/masculine.json', 'r')
englishMasculineWords = json.load(englishMasculineWordsList)
englishFeminineWordsList = open('./data/gendered_words_lists/en/feminine.json', 'r')
englishFeminineWords = json.load(englishFeminineWordsList)

In [16]:
# Count the number of gendered words in each job advertisement
dataDirectory = './data/job_advertisements/en/cleaned/'
# get the list of file names
files = [f for f in listdir(dataDirectory) if isfile(join(dataDirectory, f)) and ('failed' not in f)]

# declare the dataframe object to save the results
df = pd.DataFrame(index=[], columns=['index', 'occupation', 'masculineWordCount', 'feminineWordCount', 'wordCount'])
dfByOccupation = pd.DataFrame(index=[], columns=['index', 'occupation', 'masculineWordCount', 'feminineWordCount', 'wordCount', 'number'])
genderedWordDf = pd.DataFrame(index=[], columns=['index', 'word', 'occupation'])

occupationIndex = 0
for file in files:
    occupationIndex += 1
    # open job advertisements file
    jsonOpen = open(dataDirectory + file, 'r')
    jobAdvertisements = json.load(jsonOpen)
    occupation = file.replace('.json', '')
    
    masculineWordCounterPerOccupation = 0
    feminineWordCounterPerOccupation = 0
    wordCounterPerOccupation = 0

    advertisementIndex = 1
    for jobAdvertisement in jobAdvertisements:
        all_sentences = jobAdvertisement['all_sentences']
        masculineWordCounterPerAdvertisement = 0
        feminineWordCounterPerAdvertisement = 0
        counter = 0
        for word in all_sentences.split(' '):
            counter += 1
            for englishMasculineWord in englishMasculineWords:
                if englishMasculineWord.lower() == word or ( '*' in englishMasculineWord and englishMasculineWord.lower().replace('*', '') in word):
                    masculineWordCounterPerAdvertisement += 1
                    wordData = pd.DataFrame({
                        'index': counter,
                        'word': englishMasculineWord,
                        'occupation': occupation,
                    }, index=[str(counter)])
                    genderedWordDf = pd.concat([genderedWordDf, wordData])
                    break
            for englishFeminineWord in englishFeminineWords:
                if englishFeminineWord.lower() == word or ( '*' in englishFeminineWord and englishFeminineWord.lower().replace('*', '') in word):
                    feminineWordCounterPerAdvertisement += 1
                    wordData = pd.DataFrame({
                        'index': counter,
                        'word': englishFeminineWord,
                        'occupation': occupation,
                    }, index=[str(counter)])
                    genderedWordDf = pd.concat([genderedWordDf, wordData])
                    break
        record =  pd.DataFrame({
            'index': advertisementIndex,
            'occupation': occupation,
            'masculineWordCount': masculineWordCounterPerAdvertisement,
            'feminineWordCount': feminineWordCounterPerAdvertisement,
            'wordCount': counter
        }, index=[str(advertisementIndex)])
        df = pd.concat([df, record])
        masculineWordCounterPerOccupation += masculineWordCounterPerAdvertisement
        feminineWordCounterPerOccupation += feminineWordCounterPerAdvertisement
        wordCounterPerOccupation += counter
        advertisementIndex += 1

    record =  pd.DataFrame({
        'index': occupationIndex,
        'occupation': occupation,
        'masculineWordCount': masculineWordCounterPerOccupation,
        'feminineWordCount': feminineWordCounterPerOccupation, 
        'wordCount': wordCounterPerOccupation,
        'number': int(advertisementIndex - 1)
    }, index=[str(occupationIndex)])
    dfByOccupation = pd.concat([dfByOccupation, record])

df.to_csv('./result/english_result.csv', index=False)
dfByOccupation.to_csv('./result/english_result_per_occupation.csv', index=False)
genderedWordDf.to_csv('./result/english_gendered_words_in_advertisements.csv', index=False)